In [46]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
import re
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, LSTM
from keras.callbacks import ModelCheckpoint

In [2]:
data = pd.read_feather('lyrics/alvar_scrape.ft')

In [3]:
data.head()

,artist,title,lyrics
0,SIP€LGA 14,Kanepimees,"[Intro]\nHallo, kes on?\nKarmo\nKarmo!\nLase s..."
1,SIP€LGA 14,Kanepimees / räpasemees,[Chorus]\nKanepimees\nOi oi\nPaar kedo veel\nO...
2,SIP€LGA 14,MINE PUCCI,[Chorus]\nKui ment on ukse taga siis sa lihtsa...
3,SIP€LGA 14,AITABKÜLL,[Verse 1: SIP€LGA 14]\nPesuehtne rahustite apa...
4,SIP€LGA 14,LUUADAINEDRAHA,[Intro]\nS I P E L G A\n(hehehe jou)\n\n[Verse...


In [35]:
raw_text = ""
for index, row in data.iterrows():
    raw_text += re.sub(r"\n", " ", row['lyrics']).lower()

In [36]:
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))

In [37]:
n_chars = len(raw_text)
n_vocab = len(chars)

In [38]:
print('Chars : ' , n_chars)
print('Vocab : ', n_vocab)

Total Characters :  1405412
Total Vocab :  137


In [40]:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    seq_in  = raw_text[i:i+seq_len]
    seq_out = raw_text[i + seq_len]
    data_X.append([chars_int[char] for char in seq_in])
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Num Patterns : ', n_patterns)

Total Patterns :  1405312


In [47]:
X = np.reshape(data_X , (n_patterns, seq_len, 1))
X = X/ float(n_vocab)
y = np_utils.to_categorical(data_y)

In [48]:
LSTM_layer_num = 4
layer_size = [256,256,256,256]

In [49]:
model = Sequential()

In [50]:
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))

In [52]:
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))

In [53]:
model.add(Flatten())

In [54]:
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [55]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 137)               3507337   
_________________________________________________________________
activation (Activation)      (None, 137)               0

In [56]:
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [57]:
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])


Epoch 1/30
  38/8784 [..............................] - ETA: 11:33:33 - loss: 3.3072

KeyboardInterrupt: 

In [58]:
#hiljem path ette anda kui ära treeninud
weights_file = ''
model.load_weights(weights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

OSError: Unable to open file (unable to open file: name = './models/Weights-LSTM-improvement-004-2.49538-bigger.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")

generated_characters = 500

for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

" u märg nagu  evelini suudlus murran sinu naist nagu tema murrab truudust tihti kuuled mu nime, kuule "

                                                                                                                    